In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [4]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)

model.pool.register_forward_hook(layer1_hook)

In [ ]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [6]:
outputs

[tensor([[[[0.2415, 1.3763, 0.7276,  ..., 1.2936, 0.6038, 1.3411],
           [1.4400, 0.6664, 1.8422,  ..., 0.5921, 0.6779, 0.4613],
           [1.6598, 1.8536, 0.6854,  ..., 1.0215, 1.5402, 0.9231],
           ...,
           [1.6347, 0.6404, 0.5504,  ..., 0.2078, 0.3963, 0.3839],
           [1.0808, 0.1545, 0.7740,  ..., 0.6426, 0.6536, 1.1092],
           [0.4819, 0.3031, 0.3597,  ..., 0.9757, 0.7995, 1.0357]],
 
          [[0.8365, 0.0000, 0.1504,  ..., 0.1536, 0.9191, 0.8239],
           [1.3612, 1.1756, 0.7112,  ..., 1.0478, 0.6981, 1.3922],
           [0.4473, 0.4580, 1.6039,  ..., 1.6072, 0.2238, 0.0000],
           ...,
           [0.5208, 0.3197, 0.4138,  ..., 0.4547, 0.5069, 1.2709],
           [0.7386, 0.6545, 1.1708,  ..., 0.9969, 0.6994, 0.2885],
           [1.4769, 0.1775, 0.9832,  ..., 1.2577, 0.8351, 0.4186]],
 
          [[1.0176, 0.8048, 0.8544,  ..., 0.4424, 0.5881, 0.1794],
           [0.5343, 0.4969, 0.8094,  ..., 1.2897, 0.4380, 0.4798],
           [0.3216, 1.25

In [7]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [8]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [9]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


